# Notebook activity RU.01 Fitting a logistic regression model from a contingency table

This notebook activity will explain how to manipulate data given in the form of a contingency table into a form to which a logistic regression model can be fitted, focusing in particular on the Child measurements dataset.

These data were presented in the form of a contingency table in Table 20, Subsection 4.4 of the Review Unit.  This form is suitable for analysis through a log-linear model. However, let's assume that a logistic regression model with `obese` as the response variable is more suitable to answer the research question for this study. We will consider a way to manipulate the data into the required form, and then will fit a logistic regression model.

As a reminder, the variables in the Child measurements dataset are described as follows.

* `ethnicity`: the child's ethnicity, taking the six possible values Asian, Black, Mixed, White, Chinese or other, and Unknown
* `ageGroup`: the child's age, taking the two possible values 4 to 5 years and 10 to 11 years
* `obese`: whether a child is classified as being obese based on their BMI, taking the two possible values no and yes.

The data for this dataset are stored in the data frame `childMeasurements`. 

## (a) Getting started

As usual, we will start by loading the `M348` package into the session and `childMeasurements`. We will check that the data frame has loaded OK. Here, we won't use the `head()` function because we need to see the whole dataset for our manipulations later on.

### (i)

To do all of this, run the code cell below.

In [ ]:
#Load the M348 package
library(M348)
#Load the data frame
data(childMeasurements)
# Check the data frame has loaded OK
childMeasurements

## (b) Manipulating the data in order to fit a logistic model with response variable `obese`

The data in `childMeasurements` are in a form suitable for modelling with a log-linear model. Each row of data in the data frame relates to one of the cells in a contingency table: the value of the variable `count` is the number of observations which fall into the `ethnicity`, `ageGroup` and `obese` categories specified on that row. We can't, however, fit a logistic regression model to the data in this form. 

What can we do about this? Well, we can either:

* view the outcome of `obese` (taking values yes or no) for each child individually and fit a logistic regression model on these Bernoulli response variables, taking `ethnicity` and `ageGroup` as explanatory variables, or 
* we can view the number of children taking the value yes for `obese` in each `ethnicity` and `ageGroup` category as binomial response variables and fit a logistic regression model on these, again taking `ethnicity` and `ageGroup` as explanatory variables. 

Both models will give the same results.

The Bernoulli approach will require us to specify a row in the dataset for each child - with a sample size of 890,608 this seems excessive! So, let's go for the binomial approach.

### (i)

Apart from the number of children taking the value yes for `obese` in each `ethnicity` and `ageGroup` category, what else do we need to know to fit a binomial GLM?

#### Solution <a class="tocSkip">

A binomial random variable describes the number of 'successes' out of a specific number of Bernoulli trials. So, in addition to knowing the number of children who take the value yes for `obese` (that is, the number of 'successes') in each `ethnicity` and `ageGroup` category, in order to fit a binomial GLM, we need to know the number of children who take the value no for  `obese` in each `ethnicity` and `ageGroup` category.

---

###  <a class="tocSkip">

Following on from part (b)(i), our data frame for using a binomial GLM needs a row for each `ethnicity` and `ageGroup` category, and each row needs the number of children taking the value yes for `obese` in that `ethnicity` and `ageGroup` category, and the number of children taking the value no for `obese` in that category.

For example, 3537 Asian children in the 4-5 years age group take value yes for `obese`, and 33241 Asian children in the 4-5 years age group take value no for `obese`. So, to fit a binomial GLM, we need both of these numbers in a row corresponding to Asian children in the 4-5 years age group. 

To do this, we first need to think about how the data are currently arranged, and how we need them to be arranged for a binomial GLM.

In the current data frame (suitable for modelling with a log-linear model):

* Row 1 relates to category Asian for `ethnicity`, 4-5 years for `ageGroup` and yes for `obese`. The value of `count` in row 1 is 3537, and so there are 3537 'successes' (when `obese` is yes) for the category 'Asian for `ethnicity`, 4-5 years for `ageGroup`'.

* Row 7 relates to category Asian for `ethnicity`, 4-5 years for `ageGroup` and no for `obese`. The value of `count` in row 7 is 33241, and so there are 33241 'failures' (when `obese` is no) for the category 'Asian for `ethnicity`, 4-5 years for `ageGroup`'. 

In a data frame suitable for modelling with a binomial GLM, we need both the number of 'successes' and the number of 'failures' for the category 'Asian for `ethnicity`, 4-5 years for `ageGroup`' on the *same* row. To do this, we can create a new variable - `obeseNo` say - so that the value of `count` from row 7, becomes the value of `obeseNo` in row 1. 

Likewise, rows 2 and 8 both relate to the same `ethnicity` and `ageGroup` category (Black for `ethnicity` and 4-5 years for `ageGroup`), but `obese` is yes in row 2 and is no in row 8. So, 

* the value of `count` in row 2 is the number of 'successes' for the category 'Black for `ethnicity`, 4-5 years for `ageGroup`', and 

* the value of `count` in row 8 is the number of 'failures' for the category 'Black for `ethnicity`, 4-5 years for `ageGroup`'.

So, in our data frame suitable for modelling a binomial GLM, we need the value of `count` from row 8 to become the value of `obeseNo` in row 2. 

We can continue in this way so that, for each `ethnicity` and `ageGroup` category where `obese` is yes, the associated row of data has the variables `count` giving the number of 'successes' for that category, and `obeseNo` giving the number of 'failures' in that category.

### (ii)

Run the code in the next code cell to add the new variable `obeseNo`. What has the code achieved?

In [ ]:
for (i in 1:6){
    childMeasurements$obeseNo[i] <- childMeasurements$count[6 + i]
    childMeasurements$obeseNo[12 + i] <- childMeasurements$count[18 + i]
}

childMeasurements

#### Solution <a class="tocSkip">

Running the code cell produces the following output.

In [3]:
for (i in 1:6){
    childMeasurements$obeseNo[i] <- childMeasurements$count[6 + i]
    childMeasurements$obeseNo[12 + i] <- childMeasurements$count[18 + i]
}

childMeasurements

count  ethnicity     ageGroup       obese obeseNo
1    3537 Asian         4 to 5 years   yes    33241 
2    2607 Black         4 to 5 years   yes    14800 
3    2194 Mixed         4 to 5 years   yes    19171 
4   24240 White         4 to 5 years   yes   227637 
5    1062 Chinese/other 4 to 5 years   yes     9830 
6    5764 Unknown       4 to 5 years   yes    55387 
7   33241 Asian         4 to 5 years   no     33241 
8   14800 Black         4 to 5 years   no     33241 
9   19171 Mixed         4 to 5 years   no     33241 
10 227637 White         4 to 5 years   no     33241 
11   9830 Chinese/other 4 to 5 years   no     33241 
12  55387 Unknown       4 to 5 years   no     33241 
13  12463 Asian         10 to 11 years yes    36837 
14   7468 Black         10 to 11 years yes    17646 
15   5413 Mixed         10 to 11 years yes    17766 
16  60565 White         10 to 11 years yes   251319 
17   3098 Chinese/other 10 to 11 years yes     9522 
18  14355 Unknown       10 to 11 years yes    54686 
19  36837 Asian         10 to 11 years no     33241 
20  17646 Black         10 to 11 years no     33241 
21  17766 Mixed         10 to 11 years no     33241 
22 251319 White         10 to 11 years no     33241 
23   9522 Chinese/other 10 to 11 years no     33241 
24  54686 Unknown       10 to 11 years no     33241

After running the code, there is an extra column `obeseNo`, containing the number of 'failures' for each `ethnicity` and `ageGroup` category.

Notice that for rows 7 to 12 and 19 to 24, where `obese` is no, the value of `obeseNo` is simply 33241 (the same value of `obeseNo` for the first row). 

---

###  <a class="tocSkip">

The command ```for (i in 1:6){}``` creates a for loop. (Loops were introduced in Notebook activities A2.3 and B1.4.) All commands inside the curly brackets ```{}``` will be executed for values 1, 2, ..., 6 of the index `i`. Here, a new column called `obeseNo` is created and added to the data frame. In the first step, when `i` equals 1, the first value in the new column `obeseNo` is assigned the (6+i)th, or 7th, value in the column `count`, and the 13th value of `obeseNo` is assigned the 19th value in `count`. Then `i` becomes 2, and the 2nd and the 14th values of `obeseNo` are assigned, and so on.

This means that for all rows relating to groups of children where the value of `obese` is equal to yes, we now have the matching numbers of children taking values yes and no together in a row. For example, in the first row, you can find both the number of children taking the value yes for `obese` (in the column labelled `count`), and the number of children taking the value no for `obese` (in the column labelled `obeseNo`) for 'Asian for `ethnicity`, 4-5 years for `ageGroup`' category.

For the rows where the value of `obese` is equal to no, the code has simply put the number 33241 (the number of 'failures' on the first row) into the `obeseNo` column. These rows are now redundant, since all of the required information is now in rows 1 to 6 and 13 to 18 (where `obese` is yes).

So, we can merge  rows 1 to 6 and 13 to 18 into a new data set, while also removing the `obese` column, which is obsolete for the binomial GLM.

### (iii)

Run the code in the next cell. This will give you a dataset for which we can fit a binomial GLM.

In [ ]:
childMeasurementsNew <- rbind(childMeasurements[1:6, -4],
                              childMeasurements[13:18, -4])
childMeasurementsNew

What has the code achieved? Well, we have created a new data frame called `childMeasurementsNew`. The `rbind` command has combined rows 1 to 6 and 13 to 18 of the data frame `childMeasurements`. The argument `-4` means that the fourth column (in this case the column for `obese`) is removed. For example, ```childMeasurements[1:6,-4]``` means rows 1 to 6 of the data frame `childMeasurements` without the fourth column.

## (c) Fitting a binomial GLM with response variable `obese`

We're now (nearly!) ready to fit a binomial GLM to data in the new data frame `childMeasurementsNew`.
Before we do so though, recall from Notebook activity 7.5 that when we have a binomial dataset, the response is specified by a *matrix*, where the first column of the matrix contains the 'successes' and the second column of the matrix contains the 'failures'.

### (i)

Run the code cell below to create the response matrix `obeseMatrix` from the data frame `childMeasurementsNew`. 

In [5]:
childMeasurementsNew$obeseMatrix <- cbind(childMeasurementsNew$count, 
                                          childMeasurementsNew$obeseNo)

Running the code cell will create the required response matrix, but won't produce any output.

### (ii)

Run the code cell below to fit a logistic regression model with response variable `obese` and factors `ethnicity` and `ageGroup` as explanatory variables, to the new data frame `childMeasurementsNew`. 

Confirm that this fitted model has the same residual deviance and degrees of freedom as were stated in part (b) of Activity 33 (Subsection 5.2.1).

In [ ]:
fitBinomial <- glm(obeseMatrix ~ ethnicity + ageGroup, family = binomial, 
                   data = childMeasurementsNew)
summary(fitBinomial)

#### Solution <a class="tocSkip">

From the output, the residual deviance is 302.78  on  5  degrees of freedom. These values are indeed the same as those given in part (b) of Activity 33. 